In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from os import listdir
import json

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [5]:
from google.colab import drive
drive.mount('./drive', force_remount=True)

path_prefix = './drive/My Drive/CS48002-NetworkScience/'

Mounted at ./drive


In [6]:
filename = "Google-Playstore.csv"
df = pd.read_csv(join(path_prefix, filename))

In [113]:
play_store = df.nlargest(1001, 'Maximum Installs')

In [99]:
play_store.shape

(1000, 24)

In [100]:
# Upload Permissions Dataset
filename = "permissions_dataset.csv"
permissions = pd.read_csv(join(path_prefix, filename))

In [10]:
permissions.shape

(2292693, 3)

In [114]:
permissions.rename(columns = {'appId':'App Id', 'allPermissions':'All Permissions'}, inplace = True)

In [102]:
permissions.drop(columns=['appName'], inplace=True)

In [115]:
# Merge two datasets
data_df = pd.merge(permissions, play_store, on='App Id')

In [116]:
data_df.shape

(1000, 25)

In [117]:
data_df.head()

,App Id,All Permissions,App Name,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,com.google.android.apps.messaging,[{'permission': 'modify or delete the contents...,Messages,Communication,4.4,5305758.0,"1,000,000,000+",1.000000e+09,1931517750,True,...,http://www.google.com/mobile/android/,apps-help@google.com,"Nov 12, 2014","Jun 10, 2021",Everyone,http://www.google.com/policies/privacy,False,False,False,2021-06-16 06:11:32
1,com.cardinalblue.piccollage.google,[{'permission': 'modify or delete the contents...,PicCollage - Create & Celebrate!,Photography,4.6,1574385.0,"50,000,000+",5.000000e+07,89402935,True,...,http://pic-collage.com/,support@piccollage.com,"Jun 7, 2012","Jun 07, 2021",Everyone,http://cardinalblue.com/privacy,True,True,False,2021-06-16 05:29:28
2,com.melesta.coffeeshop,"[{'permission': 'view Wi-Fi connections', 'typ...",My Cafe — Restaurant Game. Serve & Manage,Casual,4.5,258455.0,"50,000,000+",5.000000e+07,78114532,True,...,https://legal.melsoft-games.com/,support@melsoft-games.mail.helpshift.com,"May 25, 2016","Jun 10, 2021",Teen,https://www.moonactive.com/privacy-policy/,True,True,False,2021-06-16 10:41:36
3,tunein.player,[{'permission': 'read the contents of your USB...,"TuneIn Radio: News, Sports & Online Music Stat...",Music & Audio,4.5,1892962.0,"100,000,000+",1.000000e+08,158758429,True,...,http://tunein.com/support/android/,android-support@tunein.com,"Sep 2, 2010","Jun 14, 2021",Teen,http://tunein.com/policies/privacy/,True,True,True,2021-06-16 13:49:26
4,com.alien.shooter.galaxy.attack,"[{'permission': 'view Wi-Fi connections', 'typ...",Galaxy Attack: Alien Shooter,Arcade,4.6,404551.0,"100,000,000+",1.000000e+08,151557110,True,...,https://abigames.com.vn,feedback@abigames.com.vn,"Oct 22, 2016","Jun 12, 2021",Everyone,https://www.abigames.com.vn/policy,True,True,False,2021-06-16 12:17:52


In [118]:
for i,row in data_df.iterrows(): 
  perms=row["All Permissions"]
  if type(perms)==str:

    data_df.at[i,"All Permissions"]=perms


In [119]:
import re
for i,row in data_df.iterrows(): 
  perms=row["All Permissions"]
  listofdict=[]
  if type(perms)==str:
    #change key's surrounding '' to ""
    for idx in range(len(perms)):
      char=perms[idx]
      if char=="'" and perms[idx+1]==":":
        idx2=perms.rfind("'",0,idx-1)
        temp=list(perms)
        temp[idx]='"'
        temp[idx2]='"'
        perms=''.join(temp)

    for idx in range(len(perms)):
      char=perms[idx]
      if char=="{":
        idx2=perms.find("}",idx)
        dict_obj=perms[idx+1:idx2]
        cols=[]
        for i2 in range(len(dict_obj)):
          for ch in re.finditer(":",dict_obj):
            idcol=ch.start()
            cols.append(idcol)
        #dict_obj is a string like this now: {"permission": 'retrieve running apps', "type": 'Device & app history'}

        v1=dict_obj.find(",",cols[0])
        val1=dict_obj[cols[0]+3:v1-1]
        val2=dict_obj[cols[1]+3:-1]

        item={
            "permission":val1,
            "type":val2
        }
        listofdict.append(item)     
    data_df.at[i,"All Permissions"]=listofdict

In [122]:
data_df.to_csv(path_prefix+"network_dataset.csv")

In [123]:
from google.colab import files
files.download(path_prefix+"network_dataset.csv") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [124]:
data_df.sample(3)

,App Id,All Permissions,App Name,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
143,com.billiards.city.pool.nation.club,"[{'permission': 'view Wi-Fi connections', 'typ...",Pooking - Billiards City,Sports,4.4,2671414.0,"100,000,000+",100000000.0,280522436,True,...,http://billiardscity.net/,zhangli332211@gmail.com,"May 15, 2017","Jun 16, 2021",Everyone,https://sites.google.com/site/billiardscitypri...,True,False,False,2021-06-16 11:35:58
719,com.Garawell.BridgeRace,[{'permission': 'read phone status and identit...,Bridge Race,Casual,3.4,435179.0,"50,000,000+",50000000.0,77531415,True,...,https://supersonic.com/,nebibasaran@gmail.com,"Feb 13, 2021","Jun 14, 2021",Everyone,https://supersonic.com/privacy/,True,True,False,2021-06-16 05:09:04
807,com.geishatokyo.trafficrun,[{'permission': 'read the contents of your USB...,Traffic Run!,Action,4.0,245215.0,"50,000,000+",50000000.0,92917805,True,...,http://www.geishatokyo.com/,hc+trafficrun@geishatokyo.com,"Apr 5, 2019","May 26, 2021",Everyone,https://www.geishatokyo.com/#!/privacy.html,True,True,False,2021-06-15 22:17:55


In [126]:
def permissionSimilarity(id1, id2):
  row1=data_df[data_df["App Id"]==id1]
  row2=data_df[data_df["App Id"]==id2]

  perms1=list(row1["All Permissions"])[0]
  perms2=list(row2["All Permissions"])[0]

  common_perms = set()

  for per1 in perms1:
    for per2 in perms2:
      if per1["permission"]==per2["permission"]:
        common_perms.add(per1["permission"])
  
  if len(perms1)+ len(perms2) == 0:
    return 0
  
  return len(common_perms) / (((len(perms1)+len(perms2))/2))

In [129]:
import csv

edgeList = []

for i, row1 in data_df.iterrows():
  for j,row2 in data_df.iterrows():
    if row1["App Id"] != row2["App Id"] and j > i:
      edgeWeight = permissionSimilarity(row1["App Id"], row2["App Id"])
      if edgeWeight > 0: 
        edge = ((row1["App Id"], row2["App Id"], edgeWeight))
        edgeList.append(edge)
  print(i)
  
with open(path_prefix+'network_edges', 'w') as f: 
  write = csv.writer(f)
  write.writerows(edgeList)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [130]:
files.download(path_prefix+"network_edges") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [133]:
import networkx as nx
appNet = nx.Graph()
appNet.add_weighted_edges_from(edgeList)

In [141]:
import community.community_louvain as community_louvain

communities = community_louvain.best_partition(appNet)

uniq = []

ValueError: ignored

In [142]:
for key in communities:
  if communities[key] not in uniq:
    uniq.append(communities[key])
print(uniq)


[1, 0]


In [140]:
with open(path_prefix+'app_communities.txt', 'w') as f: 
  f.write(json.dumps(communities))

files.download(path_prefix+"app_communities.txt") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def nodePropertiesMethod(network):
  nodeProperties = dict()

  for k,v in dict(network.out_degree()).items():
    if k not in nodeProperties:
      nodeProperties[k] = dict()
    nodeProperties[k]['outdegree'] = v

  for k,v in dict(network.in_degree()).items():
    if k not in nodeProperties:
      nodeProperties[k] = dict()
    nodeProperties[k]['indegree'] = v

  for k,v in dict(nx.eigenvector_centrality(network)).items():
    if k not in nodeProperties:
      nodeProperties[k] = dict()
    nodeProperties[k]['eigvec-centrality'] = v

  for k,v in dict(nx.betweenness_centrality(network)).items():
    if k not in nodeProperties:
      nodeProperties[k] = dict()
    nodeProperties[k]['betweenness-centrality'] = v

  for k,v in dict(nx.closeness_centrality(network)).items():
    if k not in nodeProperties:
      nodeProperties[k] = dict()
    nodeProperties[k]['closeness-centrality'] = v

  for k,v in dict(nx.clustering(network)).items():
    if k not in nodeProperties:
      nodeProperties[k] = dict()
    nodeProperties[k]['clustering'] = v

  for k,v in dict(nx.average_neighbor_degree(network)).items():
    if k not in nodeProperties:
      nodeProperties[k] = dict()
    nodeProperties[k]['avg.neighbor deg.'] = v

  nodePropertiesDf = pd.DataFrame.from_dict(nodeProperties, orient='index')
  return nodePropertiesDf

permsDf=nodePropertiesMethod(appNet)
permsDf

In [ ]:
def stats_of_df(df,name):

  for col in df.columns:

    stat=df.sort_values(by=[col],ascending=False)
    print("{} with the highest {}:".format(name,col),stat.head(1)[col],"\n")

stats_of_df(permsDf,"App")

In [ ]:
nx.draw(appNet,node_size=15,node_shape="o",node_color="#1DA1F2")

In [ ]:
def graphProperties(G, name): #a function to calculate the graph properties

  print("graph properties for",name,"\n")

  nodes=nx.number_of_nodes(G)
  edges=nx.number_of_edges(G)

  print("number of nodes in {} is".format(name),nodes)
  print("number of edges in {} is".format(name),edges,"\n")

  if nx.is_connected(G):
    print("largest connected component is the graph itself, having the following nodes:")
    largest_cc = max(nx.connected_components(G), key=len)
    print(largest_cc,"\n")
    print("radius of the {} graph is:".format(name),nx.radius(G))
    print("diameter of the {} graph is:".format(name),nx.diameter(G),"\n")
  else:
      diams = []
      components  = (G.subgraph(c).copy() for c in nx.connected_components(G))
      for c in components:
          diams.append(nx.diameter(c))
      rads = []

      print("Connected subgraphs in graph:")
      for c in components:
        print(c)
        rads.append(nx.radius(c))
      largest_cc = max(nx.connected_components(G), key=len)
      print(largest_cc)
      print("maximum diameter among the connected subgraphs of {}:".format(name),max(diams))
      print("maximum radius among the connected subgraphs of {}:".format(name),max(rads),"\n")
  
  cliques=nx.graph_number_of_cliques(G)

  density = G.number_of_edges()/(G.number_of_nodes()*(G.number_of_nodes()-1))
  print("density of {} is:".format(name),density,"\n")
  print("number of cliques in {} is:".format(name),cliques,"\n")

  print("---------------------------------------------------------------")

In [ ]:
graphProperties(appNet,"permission network")